<a href="https://colab.research.google.com/github/ptran1203/face_recognition/blob/master/face_recognition.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive, output
data_loaded = False
# drive.mount('/content/drive')
BASE_DIR = "/content/drive/My Drive/face_recognition"
!rm -rf '/content/face_recognition'
!git clone https://github.com/ptran1203/face_recognition
!pip install face_recognition
%cd face_recognition

In [ ]:
from dataloader import *
from model import FaceModel, AutoEncoder
from utils import *
from face_localization import *

import tensorflow as tf

data_path = "./dataset/"

class FaceRecognizer(FaceModel):
    pass

class DataGen(DataGenerator):
    pass

rst = 128
data_gen = DataGen(data_path, 128, rst, split_option=1)
fmodel = FaceRecognizer(rst, len(data_gen.classes), lr=1e-5)

In [ ]:
import pandas as pd

def plot_dist(data, figsize=(20, 10)):
    count = dict(Counter(data))
    count = list(count.items())
    count = sorted(count, key=lambda x: x[1])
    df = pd.DataFrame(count)
    df.columns = ["Identities", "Count"]
    plt.figure(figsize=figsize)
    chart = sns.barplot(
        data=df,
        x='Identities',
        y='Count')

    chart.set_xticklabels(chart.get_xticklabels(), rotation=45,
                          horizontalalignment='right')

plot_dist(np.concatenate([data_gen.labels, data_gen.labels_test, data_gen.labels_support]))
# plot_dist(data_gen.labels)



In [4]:
# autoencoder = AutoEncoder(0.001, 128, 128)
# autoencoder.train(data_gen, 20)
# imgs = data_gen.x[data_gen.per_class_ids[6]]
# gen = autoencoder.model.predict(imgs)
# show_images(imgs[:10],deprcs=False)
# show_images(gen[:10],deprcs=False)

# errors = []
# for i in range(len(gen)):
#     errors.append(np.mean(np.square(gen[i] - imgs[i])))

# idxs = np.where(np.array(errors) >= 0.02)[0]
# show_images(imgs[idxs][:10],deprcs=False)
# print(sorted(errors)[::-1])
# scatter_plot(imgs, [0] * len(gen),
#              autoencoder.encoder, opt='tsne', plot_img=1,
#              figsize=(15,15), image_zoom=0.6,)

In [ ]:
fmodel.train(data_gen, epochs=20)
fmodel.calculate_embeddings(data_gen.x_support, data_gen.y_support)
test_acc, pred_wrong = fmodel.evaluate(data_gen.x_test, data_gen.y_test)
print("Test accuracy: {}%".format(round(test_acc*100, 2)))
print("Incorrect prediction")
for i in range(0, len(pred_wrong), 10):
    show_images(data_gen.x_test[pred_wrong][i: i+10], denorm=True, deprcs=False)

In [ ]:
x, labels = data_gen.x_test, data_gen.labels_test
scatter_plot(x, labels,
             fmodel.embedding, opt='tsne', plot_img=0,
             figsize=(15,15), image_zoom=0.6,)

In [ ]:
x, y, labels = data_gen.x_test, data_gen.y_test, data_gen.labels_test
print(np.unique(labels))
fmodel.calculate_embeddings(x, y)

img = get_image_http('https://cdn.kstarlive.com/image/1573010198739-5.jpg')

if img is not None:
    faces, bboxs = extract_multi_faces(img)
    for face, bbox in zip(faces, bboxs):
        face = cv2.resize(face, (128,128))
        x, y, ex, ey = bbox
        pred, prob = fmodel.get_prediction(norm(np.expand_dims(face, 0)),
                                    labels)
        print(pred, prob)
        prob_str = str(int(prob*100))
        text = "{} {}%".format(pred, prob_str) \
                if prob > 0.3 else "Undefined"
        result = draw_bbox(img, (x, y, ex, ey), text,
                        (148, 148, 3))

    cv2_imshow(result)


In [9]:
import base64
import html
import io
import time

from IPython.display import display, Javascript
from google.colab.output import eval_js
import numpy as np
from PIL import Image
import cv2

def start_input():
  js = Javascript('''
    var video;
    var div = null;
    var stream;
    var captureCanvas;
    var imgElement;
    var labelElement;
    
    var pendingResolve = null;
    var shutdown = false;
    
    function removeDom() {
       stream.getVideoTracks()[0].stop();
       video.remove();
       div.remove();
       video = null;
       div = null;
       stream = null;
       imgElement = null;
       captureCanvas = null;
       labelElement = null;
    }
    
    function onAnimationFrame() {
      if (!shutdown) {
        window.requestAnimationFrame(onAnimationFrame);
      }
      if (pendingResolve) {
        var result = "";
        if (!shutdown) {
          captureCanvas.getContext('2d').drawImage(video, 0, 0, 512, 512);
          result = captureCanvas.toDataURL('image/jpeg', 0.8)
        }
        var lp = pendingResolve;
        pendingResolve = null;
        lp(result);
      }
    }
    
    async function createDom() {
      if (div !== null) {
        return stream;
      }

      div = document.createElement('div');
      div.style.border = '2px solid black';
      div.style.padding = '3px';
      div.style.width = '100%';
      div.style.maxWidth = '600px';
      document.body.appendChild(div);
      
      const modelOut = document.createElement('div');
      modelOut.innerHTML = "<span>Status:</span>";
      labelElement = document.createElement('span');
      labelElement.innerText = 'No data';
      labelElement.style.fontWeight = 'bold';
      modelOut.appendChild(labelElement);
      div.appendChild(modelOut);
           
      video = document.createElement('video');
      video.style.display = 'block';
      video.width = div.clientWidth - 6;
      video.setAttribute('playsinline', '');
      video.onclick = () => { shutdown = true; };
      stream = await navigator.mediaDevices.getUserMedia(
          {video: { facingMode: "environment"}});
      div.appendChild(video);

      imgElement = document.createElement('img');
      imgElement.style.position = 'absolute';
      imgElement.style.zIndex = 1;
      imgElement.onclick = () => { shutdown = true; };
      div.appendChild(imgElement);
      
      const instruction = document.createElement('div');
      instruction.innerHTML = 
          '<span style="color: red; font-weight: bold;">' +
          'When finished, click here or on the video to stop this demo</span>';
      div.appendChild(instruction);
      instruction.onclick = () => { shutdown = true; };
      
      video.srcObject = stream;
      await video.play();

      captureCanvas = document.createElement('canvas');
      captureCanvas.width = 512; //video.videoWidth;
      captureCanvas.height = 512; //video.videoHeight;
      window.requestAnimationFrame(onAnimationFrame);
      
      return stream;
    }
    async function takePhoto(label, imgData) {
      if (shutdown) {
        removeDom();
        shutdown = false;
        return '';
      }

      var preCreate = Date.now();
      stream = await createDom();
      
      var preShow = Date.now();
      if (label != "") {
        labelElement.innerHTML = label;
      }
            
      if (imgData != "") {
        var videoRect = video.getClientRects()[0];
        imgElement.style.top = videoRect.top + "px";
        imgElement.style.left = videoRect.left + "px";
        imgElement.style.width = videoRect.width + "px";
        imgElement.style.height = videoRect.height + "px";
        imgElement.src = imgData;
      }
      
      var preCapture = Date.now();
      var result = await new Promise(function(resolve, reject) {
        pendingResolve = resolve;
      });
      shutdown = false;
      
      return {'create': preShow - preCreate, 
              'show': preCapture - preShow, 
              'capture': Date.now() - preCapture,
              'img': result};
    }
    ''')

  display(js)
  
def take_photo(label, img_data):
  data = eval_js('takePhoto("{}", "{}")'.format(label, img_data))
  return data

def js_reply_to_image(js_reply):
    """
    input: 
          js_reply: JavaScript object, contain image from webcam

    output: 
          image_array: image array RGB size 512 x 512 from webcam
    """
    jpeg_bytes = base64.b64decode(js_reply['img'].split(',')[1])
    image_PIL = Image.open(io.BytesIO(jpeg_bytes))
    image_array = np.array(image_PIL)

    return image_array

def drawing_array_to_bytes(drawing_array):
    """
    input: 
          drawing_array: image RGBA size 512 x 512 
                              contain bounding box and text from yolo prediction, 
                              channel A value = 255 if the pixel contains drawing properties (lines, text) 
                              else channel A value = 0

    output: 
          drawing_bytes: string, encoded from drawing_array
    """

    drawing_PIL = Image.fromarray(drawing_array, 'RGB')
    iobuf = io.BytesIO()
    drawing_PIL.save(iobuf, format='png')
    drawing_bytes = 'data:image/png;base64,{}'.format((str(base64.b64encode(iobuf.getvalue()), 'utf-8')))
    return drawing_bytes

In [10]:
start_input()
label_html = 'Capturing...'
img_data = ''
count = 0 
size=128
def extract_face(image, return_bbox=False, face_scale_thres = (20, 20)):
    h, w = image.shape[:2]

    try:
        (startY, startX, endY, endX) = extract_box(image)
    except Exception as e:
        return None

    minX, maxX = min(startX, endX), max(startX, endX)
    minY, maxY = min(startY, endY), max(startY, endY)
    face = image[minY:maxY, minX:maxX].copy()
    # extract the face ROI and grab the ROI dimensions
    (fH, fW) = face.shape[:2]
    # ensure the face width and height are sufficiently large
    if fW < face_scale_thres[0] or fH < face_scale_thres[1]:
        return None

    if return_bbox:
        return face, (startX, startY, endX - startX, endY - startY)

    return face

while True:
    js_reply = take_photo(label_html, img_data)
    if not js_reply:
        break

    image = js_reply_to_image(js_reply)
    face = extract_face(image, True)
    if face is not None:
        face, bbox = face
        face = cv2.resize(face, (size, size))
        x, y, ex, ey = bbox
        pred, prob = fmodel.get_prediction(norm(np.expand_dims(face, 0)),
                                    labels)
        text = "{} {}%".format(pred, str(round(prob*100, 2)))
            
        img_array = draw_bbox(image, (x,y,ex,ey), text, (148, 148, 3))
    else:
        img_array = image

    # # print(img_array.shape)
    # img_array[:,:,3] = (img_array.max(axis = 2) > 0 ).astype(int)
    # img_data = img_array
    img_data = drawing_array_to_bytes(img_array)


<IPython.core.display.Javascript object>

MessageError: ignored